In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

from tqdm import tqdm
from shapely.ops import unary_union
tqdm.pandas()

In [2]:
EQUITY_QUINTILES = [0, 0.2985527, 0.37396513, 0.43815575, 0.52905651, 1]

In [3]:
gdf_locations = gpd.read_file('../joined-data/simplified_matches_4326_full.geo.json').to_crs(3857)
gdf_ct = gpd.read_file('../ct-data/ct-data-all.geo.json').to_crs(3857)

# Assign quintile labels based on equity index
gdf_ct['quintile'] = pd.cut(
    gdf_ct['Equity Index'], bins=EQUITY_QUINTILES, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'], include_lowest=True
)

Finding 3: Average number of agency spaces (ie. individual locations) within a census tract area (ie. completely within a census tract or no more than 800 meters away from it). We want to break this down by each of the quantiles of the equity index as well.

In [4]:
result = []

# Compute population-weighted averages for all quintiles combined and by region
all_quintiles = {
    'quintile': 'Q_all',
    'total_avg': np.average(gdf_ct['total_count_800m'], weights=gdf_ct['Popu_2021'])
}
for region in ['Peel', 'Toronto', 'York']:
    region_subset = gdf_ct[gdf_ct['region'] == region]
    all_quintiles[region] = np.average(region_subset['total_count_800m'], weights=region_subset['Popu_2021'])
result.append(all_quintiles)

# Compute population-weighted averages for each quintile and by region
for quintile in ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']:
    quintile_data = gdf_ct[gdf_ct['quintile'] == quintile]
    row = {
        'quintile': quintile,
        'total_avg': np.average(quintile_data['total_count_800m'], weights=quintile_data['Popu_2021'])
    }
    for region in ['Peel', 'Toronto', 'York']:
        region_subset = quintile_data[quintile_data['region'] == region]
        if len(region_subset) > 0:
            row[region] = np.average(region_subset['total_count_800m'], weights=region_subset['Popu_2021'])
        else:
            row[region] = np.nan
    result.append(row)

# Save results to a DataFrame and CSV
average_locations_df = pd.DataFrame(result)
average_locations_df = average_locations_df.round(4)  # Round all numbers to 4 decimal places
average_locations_df.to_csv('../report-results/3_avg_locations.csv', index=False)

average_locations_df

,quintile,total_avg,Peel,Toronto,York
0,Q_all,11.9450,3.3871,19.2289,5.1813
1,Q1,13.7170,1.3693,21.9675,2.3930
2,Q2,10.4546,2.4879,21.1966,6.1955
3,Q3,10.7674,3.5817,21.9524,5.2045
4,Q4,9.4870,4.5000,13.9897,6.2356
5,Q5,15.7680,5.0676,18.3919,5.9199


Finding 4: Percentage of agency spaces near existing and future transit stations (ie. within 800m). We want this broken down by owned (CORE) and leased (CLRE)

In [5]:
# Load transit station data
gdf_existing_stations = gpd.read_file('../transit-data/transitStops-toronto.geo.json').to_crs(3857)
gdf_future_stations = gpd.read_file('../transit-data/transitStops-toronto-future.geo.json').to_crs(3857)

# Create temporary columns for proximity to transit stations
gdf_locations['near_existing'] = gdf_locations.geometry.progress_apply(
    lambda loc: gdf_existing_stations.geometry.buffer(800).intersects(loc).any()
)
gdf_locations['near_future'] = gdf_locations.geometry.progress_apply(
    lambda loc: gdf_future_stations.geometry.buffer(800).intersects(loc).any()
)
# New column: near_any (True if near either existing or future transit)
gdf_locations['near_any'] = gdf_locations['near_existing'] | gdf_locations['near_future']

# Initialize result table
result = []
location_types = ['Own', 'Rent', 'All']
columns = ['location_type', 'existing_transit', 'future_transit', 'not_near_any_transit']

# Compute percentages for each location type
for loc_type in location_types:
    if loc_type == 'All':
        subset = gdf_locations
    else:
        subset = gdf_locations[gdf_locations['Tenure'] == loc_type]

    total_count = len(subset)
    existing_count = subset['near_existing'].sum()
    future_count = subset['near_future'].sum()
    not_near_any_count = (~subset['near_any']).sum()

    result.append({
        'location_type': loc_type,
        'existing_transit': round((existing_count / total_count) * 100, 4) if total_count > 0 else 0,
        'future_transit': round((future_count / total_count) * 100, 4) if total_count > 0 else 0,
        'not_near_any_transit': round((not_near_any_count / total_count) * 100, 4) if total_count > 0 else 0
    })

# Convert result to DataFrame
transit_proximity_df = pd.DataFrame(result, columns=columns)
transit_proximity_df.to_csv('../report-results/4_transit_proximity.csv', index=False)

transit_proximity_df

100%|██████████| 2267/2267 [00:03<00:00, 647.63it/s]


,location_type,existing_transit,future_transit,not_near_any_transit
0,Own,25.0000,17.8819,63.0208
1,Rent,25.0847,23.3220,61.2203
2,All,25.9815,22.0997,61.3586


Finding 5: Percentage of agency spaces which are rented (CLRE), broken down by municipality and equity index quintile

In [6]:
# Initialize result table
result = []
quintiles = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']
columns = ['quintile', 'total_pct', 'Peel', 'Toronto', 'York']

# Compute percentages for each quintile and region
for quintile in tqdm(quintiles):
    subset = gdf_ct[gdf_ct['quintile'] == quintile]
    row = {'quintile': quintile}
    
    # Compute total percentage for all locations
    total_locations = gdf_locations[gdf_locations.geometry.intersects(
        unary_union(subset.geometry.buffer(800))
    )]
    rental_locations = total_locations[total_locations['Tenure'] == 'Rent']
    row['total_pct'] = round((len(rental_locations) / len(total_locations)) * 100, 4) if len(total_locations) > 0 else 0

    # Compute percentages for each region
    for region in ['Peel', 'Toronto', 'York']:
        region_subset = subset[subset['region'] == region]
        if len(region_subset) > 0:
            region_locations = gdf_locations[gdf_locations.geometry.intersects(
                unary_union(region_subset.geometry.buffer(800))
            )]
            region_rental_locations = region_locations[region_locations['Tenure'] == 'Rent']
            row[region] = round((len(region_rental_locations) / len(region_locations)) * 100, 4) if len(region_locations) > 0 else 0
        else:
            row[region] = 0

    result.append(row)

# Convert result to DataFrame
rental_percentages_df = pd.DataFrame(result, columns=columns)
rental_percentages_df.to_csv('../report-results/5_rental_percentages.csv', index=False)

rental_percentages_df

100%|██████████| 5/5 [00:10<00:00,  2.08s/it]


,quintile,total_pct,Peel,Toronto,York
0,Q1,59.5331,83.6735,57.6404,64.4444
1,Q2,65.3944,84.9057,61.0390,72.2467
2,Q3,65.2113,85.9155,60.4227,73.5772
3,Q4,64.0496,85.6164,60.2972,66.6667
4,Q5,59.8024,78.6517,57.7645,71.9298
